In [43]:
import pandas as pd
import numpy as np
import re
from itertools import combinations

# Muat dataset resep dengan penanganan error
recipes_df = pd.read_csv('/content/dataset_gento.csv')

# Validasi kolom dataset
required_columns = ['Title', 'Kalori', 'Ingredients', 'Steps', 'Loves']
missing_columns = [col for col in required_columns if col not in recipes_df.columns]
if missing_columns:
    raise KeyError(f"Kolom berikut tidak ditemukan di dataset: {missing_columns}")

# Pastikan kolom 'Ingredients' berisi string
recipes_df['Ingredients'] = recipes_df['Ingredients'].astype(str)

# Pastikan kolom 'Loves' berisi numerik, isi NaN dengan 0
recipes_df['Loves'] = pd.to_numeric(recipes_df['Loves'], errors='coerce').fillna(0)

# Hapus duplikasi resep berdasarkan 'Title' di dataset awal
recipes_df = recipes_df.drop_duplicates(subset=['Title'])

# Kamus pemetaan untuk pemisahan entitas bahan
ingredient_mapping = {
    'telur ayam': 'telur',
    'kaldu ayam': 'kaldu',
    'ayam': 'ayam'
}

# Fungsi untuk memisahkan entitas bahan berdasarkan kamus pemetaan
def preprocess_ingredients(ingredients):
    processed_ingredients = []
    for ing in ingredients:
        ing_lower = ing.strip().lower()
        matched = False
        for key, value in ingredient_mapping.items():
            if ing_lower == key:
                processed_ingredients.append(value)
                matched = True
                break
        if not matched:
            processed_ingredients.append(ing_lower)
    return processed_ingredients

# Fungsi untuk membersihkan dan memetakan bahan dalam dataset
def clean_and_map_ingredients(ingredients):
    # Hapus nomor urut seperti "1.", "2.", dll.
    ingredients = re.sub(r'^\d+\.\s*', '', ingredients, flags=re.MULTILINE)
    # Pisahkan bahan berdasarkan koma atau tanda lain
    ingredient_list = [item.strip().lower() for item in ingredients.split(',')]
    # Proses pemetaan entitas
    mapped_ingredients = []
    for item in ingredient_list:
        # Hapus label seperti "Bumbu :", "Pelengkap:", dll.
        item = re.sub(r'^(bumbu|pelengkap)\s*:\s*', '', item)
        # Terapkan pemetaan
        for key, value in ingredient_mapping.items():
            item = item.replace(key, value)
        mapped_ingredients.extend([sub_item.strip() for sub_item in item.split('dan') if sub_item.strip()])
    return ', '.join(mapped_ingredients)

# Terapkan pembersihan dan pemetaan ke kolom 'Ingredients' di dataset
recipes_df['Mapped_Ingredients'] = recipes_df['Ingredients'].apply(clean_and_map_ingredients)

# Fungsi untuk menghitung skor prioritas berdasarkan bahan input
def calculate_priority_score(recipe_ingredients, priority_ingredients):
    recipe_items = [item.strip() for item in recipe_ingredients.split(', ')]
    matched_ingredients = set()
    for priority_ing in priority_ingredients:
        for item in recipe_items:
            if priority_ing in item:
                matched_ingredients.add(priority_ing)
                break
    return len(matched_ingredients)

# Fungsi rekomendasi resep dengan prioritas bahan input
def recommend_recipes(ingredients, recipes_df, max_ingredients=5):
    if not ingredients or not all(isinstance(ing, str) and ing.strip() for ing in ingredients):
        return "Input bahan tidak valid atau kosong."

    # Proses input bahan dengan pemetaan
    processed_ingredients = preprocess_ingredients(ingredients)
    result = []

    # Batasi maksimal 5 bahan
    processed_ingredients = processed_ingredients[:max_ingredients]
    priority_ingredients = processed_ingredients  # Bahan input menjadi prioritas

    # Hitung skor prioritas untuk setiap resep
    recipes_df['Priority_Score'] = recipes_df['Mapped_Ingredients'].apply(
        lambda x: calculate_priority_score(x, priority_ingredients)
    )

    # Urutkan berdasarkan skor prioritas dan Loves, hilangkan duplikat
    matching_recipes = recipes_df[recipes_df['Priority_Score'] > 0].sort_values(
        by=['Priority_Score', 'Loves'], ascending=[False, False]
    ).drop_duplicates(subset=['Title'])

    if not matching_recipes.empty:
        # Ambil hingga 3 resep teratas tanpa duplikasi
        top_recipes = matching_recipes.head(3)
        result.extend(top_recipes[['Title', 'Kalori', 'Ingredients', 'Steps']].to_dict('records'))
    else:
        result.append("Tidak ada resep yang ditemukan untuk kombinasi bahan tersebut.")

    return result, processed_ingredients

# Fungsi untuk memecah Ingredients dan Steps menjadi daftar
def format_recipe_details(recipe):
    ingredients_list = [item.strip() for item in recipe['Ingredients'].split(',')]
    steps_text = recipe['Steps']
    # Pisahkan langkah berdasarkan pola angka dan titik (termasuk angka dua digit)
    steps_text = re.sub(r'(\d+\.\s)', r'\n\1', steps_text)
    steps_list = [step.strip() for step in steps_text.split('\n') if step.strip() and re.match(r'^\d+\.\s', step)]
    seen_steps = set()
    unique_steps = []
    for step in steps_list:
        if step not in seen_steps:
            seen_steps.add(step)
            unique_steps.append(step)
    return ingredients_list, unique_steps

# Inputan manual
manual_input = "tahu tempe"
ingredients = manual_input.split()
result, processed_ingredients = recommend_recipes(ingredients, recipes_df)

if isinstance(result, str):
    print(result)
else:
    print(f"Hasil pencarian untuk bahan {', '.join(processed_ingredients)}:")
    for item in result:
        if isinstance(item, str):
            print(item)
        else:
            print(f"\n{item['Title']}")
            print(f"Kalori: {item['Kalori']}")
            print("**Ingredients**:")
            ingredients_list, steps_list = format_recipe_details(item)
            for ing in ingredients_list:
                print(ing)
            print("**Steps**:")
            for step in steps_list:
                print(step)
            print()
        print()

Hasil pencarian untuk bahan tahu, tempe:

Orek tempe tahu
Kalori: 1076.975
**Ingredients**:
1. 1 papan tempe
potong dadu
2. 4 bh tahu
potong dadu
3. 1
5 sdm kecap manis
4. 40 ml air
5. Secukupnya garam
6. Gula
7. Minyak goreng
8. Bumbu halus :
9. 4 siung bawang merah
10. 3 siung bawang putih
11. 1 bh cabe merah besar
12. 2 bh cabe kecil (selera)
13. 3 cuma kunyit
14. 2 btr kemiri
15. 1 sdt ketumbar bubuk
16. 1/2 sdt merica bubuk
**Steps**:
1. Goreng tahu dan tempe, sisihkan
2. Tumis bumbu halus sampai matang lalu masukkan tempe dan tahunya, beri kecap, aduk rata, beri air, gula dan garam, aduk" hingga bumbu meresap. Sajikan



Sambel teri nasi tahu tempe
Kalori: 545.17
**Ingredients**:
1. 1 ons Teri medan
2. tempe setengh saja
3. 2 buah tahu kuning
4. 15 biji cabe merah
5. bawang putih 2 di goreng
6. bawang merah 5 di goreng
7. daun salam
8. gula
9. Royko
10. tomat setengah
**Steps**:
1. Potong tempe dan tahu bentuk Korek dan goreng sampai kering
2. Goreng teri nasi sampai kering juga 